# OpenAPI KEY - (enter a working API key for text generation (mine ran out of daily limit)still mentioned in the code)

***Files attached - 'Updated_dataset.csv' and 'interaction_log.csv'***

***Basic A/B Test Framework***

***Mock API (with future references)***

***Documentation while wrting the code***

 # ***Point 1: Product Description Generator using GPT-3.5***

This script uses the gpt-3.5-turbo model for generating product descriptions with streaming responses. It takes user inputs for product attributes and generates a description in real-time, printing it as it is received from the model. ***Make sure to replace 'your-api-key' with your actual OpenAI API key.***

***User Inputs:*** The script prompts the user to enter product attributes: product name, product category, and product features.

***Prompt Creation:*** It creates a detailed prompt for the GPT-3.5 model to generate a product description, emphasizing the role of the AI as an "e-commerce platform creative expert."


***Streaming Response:*** The script uses streaming to generate the product description in real-time.


***Description Collection:*** It collects the generated description and prints it out.


***Data Storage: ***The product attributes and generated description are stored in a dictionary.


***CSV Saving:*** The script writes the dictionary data to a CSV file (product_descriptions.csv). If the CSV file does not exist or is empty, it writes the header first.

# ***Benefits of Using AI for Product Descriptions***

***Consistency:*** AI ensures consistent tone and style across all product descriptions.

***Efficiency:*** It significantly reduces the time required to create product descriptions.


***Scalability:*** AI can handle large volumes of products effortlessly, making it ideal for extensive catalogs.


***Personalization:*** AI can tailor descriptions to different segments, enhancing user experience and engagement.


***Creativity:*** AI can generate creative and engaging content, leveraging vast amounts of training data.

In [8]:
!pip install openai
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 26.7 MB/s eta 0:00:00


In [5]:
import csv
from openai import OpenAI

# Replace 'your-api-key' with your actual OpenAI API key
client = OpenAI(api_key='OPEN_API_KEY')

def generate_product_description():
    # Taking user inputs for product attributes
    product_name = input("Enter the product name: ")
    product_category = input("Enter the product category: ")
    product_features = input("Enter the product features: ")

    # Creating the prompt for GPT-3.5 with enhanced context
    prompt = (
        f"Act as an e-commerce platform creative expert. "
        f"Generate a compelling product description for the following details:\n"
        f"Product Name: {product_name}\n"
        f"Category: {product_category}\n"
        f"Features: {product_features}\n"
        f"Description:"
    )

    # Generating the product description using GPT-3.5 with streaming
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    # Collecting the streaming response
    description = ""
    print("\nGenerated Product Description:")
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="")
            description += content

    # Storing the data in a dictionary
    product_data = {
        "product_name": product_name,
        "product_category": product_category,
        "product_features": product_features,
        "description": description.strip()
    }

    # Writing the data to a CSV file
    csv_file = 'product_descriptions.csv'
    fieldnames = ["product_name", "product_category", "product_features", "description"]

    try:
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)

            # Write header only if the file is empty
            file.seek(0, 2)
            if file.tell() == 0:
                writer.writeheader()

            writer.writerow(product_data)
        print("\nProduct description saved to CSV file successfully.")
    except IOError as e:
        print(f"Error writing to CSV file: {e}")


Enter the product name: xix
Enter the product category: vodka
Enter the product features: bottle


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

**User Inputs:** The script prompts the user to enter product attributes.

**Prompt Creation**: It creates a prompt for the GPT-3.5 model.

**Streaming Response**: The script uses streaming to generate the product description in real-time.

**Description Collection**: It collects the generated description and prints it out.

**Data Storage:** The product attributes and generated description are stored in a dictionary.

**CSV Saving:** The script writes the dictionary data to a CSV file (product_descriptions.csv). If the CSV file does not exist or is empty, it writes the header first.

**Make sure to replace 'your-api-key' with your actual OpenAI API key.**

# A Python script that creates fake/random data for 500 users with columns 'user_id', 'item_id', and 'interaction', and saves it to a CSV file named 'updated_dataset.csv'

In [6]:
import pandas as pd
import numpy as np

# Function to create random data
def create_random_data(num_users=500, num_items=100, interactions_per_user=20):
    np.random.seed(42)  # For reproducibility

    data = []
    for user_id in range(1, num_users + 1):
        for _ in range(interactions_per_user):
            item_id = np.random.randint(1, num_items + 1)
            interaction = np.random.choice(['view', 'click', 'purchase'], p=[0.7, 0.2, 0.1])
            data.append([user_id, item_id, interaction])

    return pd.DataFrame(data, columns=['user_id', 'item_id', 'interaction'])

# Generate random data
df = create_random_data()

# Save to CSV
df.to_csv('updated_dataset.csv', index=False)
print("Random data saved to 'updated_dataset.csv'")


Random data saved to 'updated_dataset.csv'


***Function to Create Random Data:*** The create_random_data function generates random data for a specified number of users, items, and interactions per user.

***Parameters:***
num_users: Number of users (default is 500).
num_items: Number of items (default is 100).
interactions_per_user: Number of interactions per user (default is 20).
For each user, it generates a specified number of interactions.
Each interaction randomly selects an item and an interaction type (view, click, purchase) with predefined probabilities.
Generate Random Data: Calls the function to generate random data.

***Save to CSV:*** Saves the generated data to a CSV file named 'updated_dataset.csv'.

# This script ensures that you have a dataset with user interactions ready for training the recommendation model.

# Point 2: Product Recommendation System
For this part, you'll need to set up a recommendation system using a relevant dataset from Kaggle.

Let's assume we are using a dataset that includes user interactions like views, clicks, and purchases. Below is a Python script using the implicit library for collaborative filtering

In [11]:
# get df head for csv file

df = pd.read_csv('updated_dataset.csv')
print(df.head())


   user_id  item_id interaction
0        1       52    purchase
1        1       72        view
2        1       83        view
3        1       75       click
4        1      100        view


In [14]:
import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix, csr_matrix

# Load the dataset (replace 'path_to_dataset' with the actual dataset path)
data = pd.read_csv('/content/updated_dataset.csv')

# Assuming the dataset has columns: 'user_id', 'item_id', 'interaction'
user_item_interaction = data[['user_id', 'item_id', 'interaction']]

# Encode the 'interaction' column
# This creates a mapping from interaction type to a numerical value
interaction_encoder = {v: i for i, v in enumerate(user_item_interaction['interaction'].unique())}
user_item_interaction['interaction_encoded'] = user_item_interaction['interaction'].map(interaction_encoder)

# Creating a sparse matrix for user-item interactions using encoded values
sparse_user_item = coo_matrix((user_item_interaction['interaction_encoded'],
                               (user_item_interaction['user_id'], user_item_interaction['item_id'])))

# Convert the COO matrix to CSR format
sparse_user_item_csr = sparse_user_item.tocsr()

# Training the ALS model
model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=30)
model.fit(sparse_user_item.T)

def recommend_products(user_id, num_recommendations=5):
    # Get recommendations for the given user
    recommendations = model.recommend(user_id, sparse_user_item_csr, N=num_recommendations)
    recommended_item_ids = [item[0] for item in recommendations]

    print(f"Recommended Products for User {user_id}: {recommended_item_ids}")

if __name__ == "__main__":
    user_id = int(input("Enter the user ID: "))
    recommend_products(user_id)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0003845691680908203 seconds
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

Enter the user ID: 1


ValueError: user_items must contain 1 row for every user in userids

# A/B Testing Framework

Implementing an A/B testing framework involves setting up a system to compare the performance of different content generation strategies or recommendation algorithms. Here's a solution that includes tracking and analyzing user interactions.

***A/B Test Setup: Assigning users to different groups.***

**Tracking User Interactions:** Logging user interactions with the content.
Analyzing Results: Comparing the performance of different strategies.

A/B Test Setup -

*First, we'll set up the A/B test by randomly assigning users to different groups.*

In [15]:
import pandas as pd
import numpy as np

# Function to assign users to A/B test groups
def assign_ab_groups(num_users=500, group_a_ratio=0.5):
    np.random.seed(42)  # For reproducibility
    users = np.arange(1, num_users + 1)
    np.random.shuffle(users)
    split_point = int(num_users * group_a_ratio)
    group_a = users[:split_point]
    group_b = users[split_point:]
    return group_a, group_b

group_a, group_b = assign_ab_groups()
print(f"Group A: {len(group_a)} users")
print(f"Group B: {len(group_b)} users")


Group A: 250 users
Group B: 250 users


# ***Tracking User Interactions***

We will create a function to log user interactions. Each interaction will be logged with the user ID, item ID, interaction type, and the group they belong to.

In [16]:
import random
from datetime import datetime

# Function to simulate user interactions
def log_interactions(num_interactions=1000):
    interactions = []
    for _ in range(num_interactions):
        user_id = random.choice(np.concatenate((group_a, group_b)))
        item_id = np.random.randint(1, 101)
        interaction = np.random.choice(['view', 'click', 'purchase'], p=[0.7, 0.2, 0.1])
        group = 'A' if user_id in group_a else 'B'
        timestamp = datetime.now().isoformat()
        interactions.append([user_id, item_id, interaction, group, timestamp])
    return pd.DataFrame(interactions, columns=['user_id', 'item_id', 'interaction', 'group', 'timestamp'])

interaction_log = log_interactions()
interaction_log.to_csv('interaction_log.csv', index=False)
print("User interactions logged and saved to 'interaction_log.csv'")


User interactions logged and saved to 'interaction_log.csv'


# ***Analyzing Results***

We will analyze the results to compare the performance of the different strategies. For simplicity, let's assume we are measuring the number of purchases.

In [17]:
# Load interaction log
interaction_log = pd.read_csv('interaction_log.csv')

# Analyze interactions by group
grouped = interaction_log.groupby('group')['interaction'].value_counts().unstack().fillna(0)

# Calculate conversion rates
conversion_rates = grouped.apply(lambda x: x / x.sum(), axis=1)
print("Conversion Rates:\n", conversion_rates)

# Additional analysis can include statistical tests to determine significance
from scipy.stats import chi2_contingency

# Create a contingency table for purchase interactions
contingency_table = pd.crosstab(interaction_log['group'], interaction_log['interaction'] == 'purchase')
chi2, p, _, _ = chi2_contingency(contingency_table)

print(f"\nChi-Square Test: chi2 = {chi2}, p-value = {p}")
if p < 0.05:
    print("The difference in purchase rates between groups is statistically significant.")
else:
    print("The difference in purchase rates between groups is not statistically significant.")


Conversion Rates:
 interaction     click  purchase      view
group                                    
A            0.209213  0.097889  0.692898
B            0.212944  0.100209  0.686848

Chi-Square Test: chi2 = 0.0002803631690226361, p-value = 0.9866408062461679
The difference in purchase rates between groups is not statistically significant.


# ***Explanation***
**A/B Test Setup:**

assign_ab_groups function randomly assigns users to Group A or Group B based on the specified ratio.

***Tracking User Interactions:***

log_interactions function simulates user interactions, logging the user ID, item ID, interaction type, group, and timestamp.

The interaction log is saved to a CSV file.

***Analyzing Results:***

Load the interaction log and group interactions by group.

Calculate and print conversion rates for each group.

Perform a Chi-Square test to determine if the difference in purchase rates between the groups is statistically significant.

***This framework can be extended to include more sophisticated logging and analysis based on specific requirements and metrics relevant***

# ***Mock API***

To create mock APIs for simulating interactions with external services such as user data retrieval and product databases, you can use Flask to create simple APIs that return realistic data. Below is a solution for creating mock APIs

In [ ]:
pip install Flask

# ***Mock API Implementation***
**app.py:**

# ***Create Flask App:***

Create a new directory for your project.
Inside the directory, create a file named app.py for the Flask application.

In [ ]:
from flask import Flask, jsonify, request
import random

app = Flask(__name__)

# Mock user data
users = [
    {"user_id": 1, "name": "Alice", "age": 30, "location": "New York"},
    {"user_id": 2, "name": "Bob", "age": 25, "location": "San Francisco"},
    {"user_id": 3, "name": "Charlie", "age": 35, "location": "Chicago"},
    # Add more mock users as needed
]

# Mock product data
products = [
    {"item_id": 1, "name": "Laptop", "category": "Electronics", "price": 999.99},
    {"item_id": 2, "name": "Smartphone", "category": "Electronics", "price": 499.99},
    {"item_id": 3, "name": "Book", "category": "Books", "price": 19.99},
    # Add more mock products as needed
]

# Mock user interactions
interactions = []

@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    user = next((user for user in users if user["user_id"] == user_id), None)
    if user:
        return jsonify(user)
    else:
        return jsonify({"error": "User not found"}), 404

@app.route('/api/products/<int:item_id>', methods=['GET'])
def get_product(item_id):
    product = next((product for product in products if product["item_id"] == item_id), None)
    if product:
        return jsonify(product)
    else:
        return jsonify({"error": "Product not found"}), 404

@app.route('/api/interactions', methods=['POST'])
def add_interaction():
    interaction = request.json
    interactions.append(interaction)
    return jsonify(interaction), 201

@app.route('/api/interactions', methods=['GET'])
def get_interactions():
    return jsonify(interactions)

if __name__ == '__main__':
    app.run(debug=True)


# ***Explanation***
**User Data Retrieval:**

The endpoint /api/users/<int:user_id> retrieves user information by user ID.
It searches for the user in the users list and returns the user's data in JSON format.

**Product Data Retrieval:**

The endpoint /api/products/<int:item_id> retrieves product information by item ID.

It searches for the product in the products list and returns the product's data in JSON format.

**Logging User Interactions:**

The endpoint /api/interactions (POST) logs user interactions.
It expects a JSON payload with interaction details and appends it to the interactions list.

**Retrieving Logged Interactions:**

The endpoint /api/interactions (GET) returns all logged interactions.

# ***Running the Flask App***

Save the app.py file.

In the terminal, navigate to the directory containing app.py.

Run the Flask application

In [ ]:
python app.py

# ***Testing the Mock APIs***
You can test the mock APIs using tools like curl or Postman.

**Example Requests:**

*Get User Data:*

In [ ]:
curl http://127.0.0.1:5000/api/users/1

*Get Product Dat*a:

In [ ]:
curl http://127.0.0.1:5000/api/products/1

*Log User Interaction:*

In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"user_id": 1, "item_id": 2, "interaction": "view"}' http://127.0.0.1:5000/api/interactions

*Get Logged Interactions:*

In [ ]:
curl http://127.0.0.1:5000/api/interactions

# ***These mock APIs provide data for testing your AI models and can be extended as needed for more complex scenarios.***